## Convergence Bispectrum Calculations

In [2]:
import numpy as np
import healpy as hp
import bispec_calculator as bispec
from helper_funcs import *

First thing we need to do is calculate $B_{\delta}(\mathbf{k}_1, \mathbf{k}_2, \mathbf{k}_3; \chi)$, which is given by:

$$B_{\delta}(\mathbf{k}_1, \mathbf{k}_2, \mathbf{k}_3; \chi) = 2F_2(\mathbf{k}_1, \mathbf{k}_2, z)P_{\delta}(\mathbf{k}_1,z)P_{\delta}(\mathbf{k_z},z) + \text{cyc. perm}$$

such that:

$$F_2(\mathbf{k}_1, \mathbf{k}_2, z) = \frac57 a(k_1, z)a(k_2, z) + \frac12 \frac{k_1^2 + k_2^2}{k_1k_2}b(k_1, z)b (k_2,z)\cos\theta + \frac27 c(k_1, z)c(k_2,z)$$

where we're using $a = b = c = 1$, giving us:

$$F_2(\mathbf{k}_1, \mathbf{k}_2, z) = \frac57 + \frac12 \frac{k_1^2 + k_2^2}{k_1k_2}\cos\theta + \frac27$$

We'll need this for every cyclic permutation, so let's code up a general function to solve for $F_2$:

In [2]:
# F2 function
def F2(k1, k2, z=0.):
    return 5/7 + 1/2 * (k1**2 + k2**2)/(k1 * k2) + 2/7

Now, since the source plan is just the CMB plane here, then we can find the comoving distance by converting the redshift of the CMB plane to comoving distance:

In [9]:
from scipy.integrate import romberg
from scipy import interpolate
# from astropy.cosmology import WMAP9 as cosmo
from astropy.constants import c
from astropy.units import u

def find_chi_from_z(z: int) -> float:
    """
    Finds the comoving distance from redshift

    Parameters:
        z (int) : redshift
    
    Returns:
        chi_from_z (float) : comoving distance
    """
    c_km = c.to(u.km/u.s).value

    def _invH(z):
        return c_km/cosmo.H(z).value

    zs = np.logspace(-2, 3, 1000)
    dcm = np.zeros_like(zs)

    for i,z in enumerate(zs):
        dcm[i] = romberg(_invH, 0, z, tol=1e-6)[0]

    chi = interpolate.interp1d(zs, dcm, bounds_error=False, fill_value = 0)

    return chi

In [10]:
def compute_scale_factor(z: int) -> float:
    """
    Finds the scale factor a(z) from z.

    Parameters:
        z (int) : redshift

    Returns:
        a (float) : scale factor
    """
    return 1 / (1 + z)